# 10x RNA-seq gene expression data (part 2a)

In part 1, we explore two examples looking at the expression of canonical neurotransmitter transporter genes and gene Tac2 in the thalamus. In this notebook, we will prepare data so that we can repeat the examples for all cells spanning the whole brain. This notebook takes ~ 5 minutes to run.

**The results from this notebook has already been cached and saved. Some of the cells might raise an error if you have not downloaded all of the data. As such, you can skip this notebook and continue with part 2b.**

You need to be connected to the internet to run this notebook and that you have downloaded the all the log2 expression matrices associated with both the WMB-10Xv2, WMB-10Xv3 and WMB-10XMulti datasets.

In [1]:
import os
import pandas as pd
import numpy as np
import anndata
import time
import json
import requests

The prerequisite for running this notebook is that the data have been downloaded to local directory maintaining the organization from the manifest.json. **Change the download_base variable to where you have downloaded the data in your system.**

In [2]:
version = '20230830'
download_base = '../../abc_download_root'

use_local_cache = False
manifest_path = 'releases/%s/manifest.json' % version

if not use_local_cache :
    url = 'https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/' + manifest_path
    manifest = json.loads(requests.get(url).text)
else :
    file = os.path.join(download_base,manifest_path)
    with open(file,'rb') as f:
        manifest = json.load(f)
    
metadata = manifest['file_listing']['WMB-10X']['metadata']

In [3]:
view_directory = os.path.join( download_base, 
                               manifest['directory_listing']['WMB-10X']['directories']['metadata']['relative_path'], 
                              'views')
cache_views = False
if cache_views :
    os.makedirs( view_directory, exist_ok=True )

In [4]:
rpath = metadata['cell_metadata']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
cell = pd.read_csv(file)
cell.set_index('cell_label',inplace=True)

### Gene expression matrices

The large 4 million cell dataset has been divided into 23 packages to make data transfer and download more efficient. Each package is formatted as annadata h5ad file with minimal metadata. In this next section, we provide example code on how to open the file and connect with the rich cell level metadata discussed above.

For each subset, there are two h5ad files one storing the raw counts and the other log normalization of it.

In [5]:
matrices = cell.groupby(['dataset_label','feature_matrix_label'])[['library_label']].count()
matrices.columns  = ['cell_count']
matrices

cell_count
dataset_label feature_matrix_label             
WMB-10XMulti  WMB-10XMulti                 1687
WMB-10Xv2     WMB-10Xv2-CTXsp             43985
              WMB-10Xv2-HPF              207281
              WMB-10Xv2-HY                99879
              WMB-10Xv2-Isocortex-1      248776
              WMB-10Xv2-Isocortex-2      249360
              WMB-10Xv2-Isocortex-3      249356
              WMB-10Xv2-Isocortex-4      248784
              WMB-10Xv2-MB                29781
              WMB-10Xv2-OLF              192182
              WMB-10Xv2-TH               130555
WMB-10Xv3     WMB-10Xv3-CB               181723
              WMB-10Xv3-CTXsp             78223
              WMB-10Xv3-HPF              181055
              WMB-10Xv3-HY               162296
              WMB-10Xv3-Isocortex-1      227670
              WMB-10Xv3-Isocortex-2      227537
              WMB-10Xv3-MB               337101
              WMB-10Xv3-MY               191746
              WMB-10Xv3-OLF               88560
              WMB-10Xv3-P                143157
              WMB-10Xv3-PAL              108046
              WMB-10Xv3-STR              283782
              WMB-10Xv3-TH               130454

### Example use cases

In this section, we explore two use cases. The first example looks at the expression of nine canonical neurotransmitter transporter genes and the second the expression of gene Tac2.

To support these use cases, we will create a smaller submatrix (all cells and 10 genes) and write to file for resue in part 2b. **Note this operation takes around 5 minutes and you might need to download more data. You can always skip this part and continue with part 2b.**

In [6]:
matrix_label = matrices.index[0][1]
dataset_label = matrices.index[0][0]

expression_matrices = manifest['file_listing'][dataset_label]['expression_matrices']
print(expression_matrices[matrix_label])

{'log2': {'files': {'h5ad': {'version': '20230830', 'relative_path': 'expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-log2.h5ad', 'url': 'https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-log2.h5ad', 'size': 89318511}}}, 'raw': {'files': {'h5ad': {'version': '20230830', 'relative_path': 'expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-raw.h5ad', 'url': 'https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-raw.h5ad', 'size': 132220015}}}}


In [7]:
rpath = expression_matrices[matrix_label]['log2']['files']['h5ad']['relative_path']
file = os.path.join( download_base, rpath)
print(file)

../../abc_download_root/expression_matrices/WMB-10XMulti/20230830/WMB-10XMulti-log2.h5ad


In [8]:
ad = anndata.read_h5ad(file,backed='r')
gene = ad.var

In [9]:
ntgenes = ['Slc17a7','Slc17a6','Slc17a8','Slc32a1','Slc6a5','Slc18a3','Slc6a3','Slc6a4','Slc6a2']
exgenes = ['Tac2']
gnames = ntgenes + exgenes
pred = [x in gnames for x in gene.gene_symbol]
gene_filtered = gene[pred]
gene_filtered

,gene_symbol
gene_identifier,
ENSMUSG00000037771,Slc32a1
ENSMUSG00000070570,Slc17a7
ENSMUSG00000039728,Slc6a5
ENSMUSG00000030500,Slc17a6
ENSMUSG00000055368,Slc6a2
ENSMUSG00000019935,Slc17a8
ENSMUSG00000025400,Tac2
ENSMUSG00000020838,Slc6a4
ENSMUSG00000021609,Slc6a3


In [10]:
# create empty gene expression dataframe
gdata = pd.DataFrame(index=cell.index,columns=gene_filtered.index)
count = 0
total_start = time.process_time()

for matindex in matrices.index :
    
    ds = matindex[0]
    mp = matindex[1]
    
    print(mp)
    
    expression_matrices = manifest['file_listing'][ds]['expression_matrices']
    rpath = expression_matrices[mp]['log2']['files']['h5ad']['relative_path']
    file = os.path.join( download_base, rpath)
    
    pred = (cell['feature_matrix_label'] == mp)
    cell_filtered = cell[pred]
    
    start = time.process_time()
    ad = anndata.read_h5ad(file,backed='r')
    exp = ad[cell_filtered.index, gene_filtered.index].to_df()
    gdata.loc[ exp.index, gene_filtered.index ] = exp
    print(" - time taken: ", time.process_time() - start)
    
    ad.file.close()
    del ad
    
    count += 1
    
    #if count > 2 :
    #    break
        
print("total time taken: ", time.process_time() - total_start)
    

WMB-10XMulti
 - time taken:  1.9622096919999983
WMB-10Xv2-CTXsp
 - time taken:  1.7534853379999973
WMB-10Xv2-HPF
 - time taken:  6.767831976
WMB-10Xv2-HY
 - time taken:  2.9769649340000015
WMB-10Xv2-Isocortex-1
 - time taken:  10.805219346000005
WMB-10Xv2-Isocortex-2
 - time taken:  12.451479411999998
WMB-10Xv2-Isocortex-3
 - time taken:  11.092032212000007
WMB-10Xv2-Isocortex-4
 - time taken:  12.191183033000002
WMB-10Xv2-MB
 - time taken:  0.9217969590000052
WMB-10Xv2-OLF
 - time taken:  6.6084268479999935
WMB-10Xv2-TH
 - time taken:  4.5150405959999915
WMB-10Xv3-CB
 - time taken:  7.923351416000003
WMB-10Xv3-CTXsp
 - time taken:  3.9744392829999953
WMB-10Xv3-HPF
 - time taken:  11.715912669999994
WMB-10Xv3-HY
 - time taken:  10.542776168000003
WMB-10Xv3-Isocortex-1
 - time taken:  17.62114658600001
WMB-10Xv3-Isocortex-2
 - time taken:  11.949673215000018
WMB-10Xv3-MB
 - time taken:  20.26076470999999
WMB-10Xv3-MY
 - time taken:  9.379144964000005
WMB-10Xv3-OLF
 - time taken:  3.1078

In [11]:
# change columns from index to gene symbol
gdata.columns = gene_filtered.gene_symbol
pred = pd.notna(gdata[gdata.columns[0]])
gdata = gdata[pred].copy(deep=True)
print(len(gdata))

4042976


In [12]:
if cache_views :
    file = os.path.join( view_directory, 'example_genes_all_cells_expression.csv')
    gdata.to_csv( file )